# 02 Preprocesamiento

En este notebook aplicamos nuestro pipeline de preprocesamiento completo y validamos:

- **Limpieza** (imputación, tratamiento de outliers, agrupación de rarezas)  
- **Feature Engineering** (nuevas variables derivadas)  
- **Encoding** de variables categóricas  

Al final comprobamos que la salida que llega al modelo sólo contiene tipos `int`, `float` o `bool`.

## 1. Imports y configuración


In [2]:
import os, sys
import pandas as pd
from sklearn import set_config
from sklearn.pipeline import Pipeline

# Asegurarnos de que 'src' está en el path
ROOT = os.path.abspath(os.path.join(".."))
if ROOT not in sys.path:
    sys.path.insert(0, ROOT)

# Para que los pipelines devuelvan DataFrames con nombres de columna
set_config(transform_output="pandas")

from src.utils import load_config
from src.full_model_pipeline import make_full_model_pipeline


## 2. Carga de datos de ejemplo

Cargamos un pequeño sample de las primeras filas del CSV de entrenamiento.

In [ ]:
# 2.1 Cargar configuración
cfg = load_config(os.path.join(ROOT, "configs", "base_config.yaml"))

# 2.2 Leer sólo 100 filas para velocidad
df_raw = pd.read_csv(os.path.join(ROOT, cfg["data_raw_path"], cfg["train_file"]), nrows=100)

# 2.3 Separar X y y
mapping = {"Cancelada": 1, "No Cancelada": 0}

y_sample = df_raw[cfg["target_column"]].map(mapping)
if y_sample.isna().any():
    raise ValueError("Encontrados valores inesperados en el target, revisar el mapeo")

X_sample = df_raw.drop(columns=cfg["id_columns"] + [cfg["target_column"]])

print("Shape X_sample:", X_sample.shape)
X_sample.head(3)

Shape X_sample: (100, 17)


,Num_Adultos,Num_Niños,Noches_Fin_Semana,Noches_Semana,Tipo_Plan_Comidas,Requiere_Estacionamiento,Tipo_Habitación_Reservada,Tiempo_Antelación,Año_Llegada,Mes_Llegada,Día_Llegada,Segmento_Mercado,Huésped_Recurrente,Num_Cancelaciones_Previas,Num_Reservas_Previas_No_Canceladas,Precio_Promedio_Por_Habitación,Num_Solicitudes_Especiales
0,2,0,0,2,Meal Plan 1,0,NaN,57.0,2018,3,4,Online,0,0,0,80.3,NaN
1,2,0,2,1,Meal Plan 1,0,Room_Type 1,282.0,2017,10,10,Offline,0,0,0,NaN,0.0
2,2,0,2,4,Not Selected,0,Room_Type 1,13.0,2018,2,5,NaN,0,0,0,79.0,0.0


## 3. Limpieza + Feature Engineering

Construimos un pipeline sólo con los dos primeros pasos (`clean` y `fe`), lo aplicamos y comprobamos columnas.

In [5]:
# Construir pipeline completo y extraer los pasos 0 y 1
full_pipe = make_full_model_pipeline(cfg)
pre_fe = Pipeline(full_pipe.steps[:2], memory=None)  # [('clean',...),('fe',...)] 

# Aplicar
X_fe = pre_fe.fit_transform(X_sample, y_sample)

# Columnas resultantes
print("Columnas tras clean + FE:")
print(X_fe.columns.tolist())
X_fe.head(3)


Dropped columns after FE: ['num_clean__Num_Adultos', 'num_clean__Num_Niños', 'num_clean__Noches_Semana', 'num_clean__Noches_Fin_Semana']
Columnas tras clean + FE:
['num_clean__Tiempo_Antelación', 'num_clean__Precio_Promedio_Por_Habitación', 'num_clean__Num_Solicitudes_Especiales', 'num_clean__Num_Adultos_miss', 'num_clean__Num_Niños_miss', 'num_clean__Noches_Semana_miss', 'num_clean__Noches_Fin_Semana_miss', 'num_clean__Tiempo_Antelación_miss', 'num_clean__Precio_Promedio_Por_Habitación_miss', 'num_clean__Num_Solicitudes_Especiales_miss', 'cat_clean__Tipo_Plan_Comidas', 'cat_clean__Tipo_Habitación_Reservada', 'cat_clean__Segmento_Mercado', 'remainder__Requiere_Estacionamiento', 'remainder__Año_Llegada', 'remainder__Mes_Llegada', 'remainder__Día_Llegada', 'remainder__Huésped_Recurrente', 'remainder__Num_Cancelaciones_Previas', 'remainder__Num_Reservas_Previas_No_Canceladas', 'Duracion_Total', 'Total_Huespedes', 'Precio_por_Persona_Noche', 'Precio_Total_Estancia', 'SameDayBooking', 'Ante

,num_clean__Tiempo_Antelación,num_clean__Precio_Promedio_Por_Habitación,num_clean__Num_Solicitudes_Especiales,num_clean__Num_Adultos_miss,num_clean__Num_Niños_miss,num_clean__Noches_Semana_miss,num_clean__Noches_Fin_Semana_miss,num_clean__Tiempo_Antelación_miss,num_clean__Precio_Promedio_Por_Habitación_miss,num_clean__Num_Solicitudes_Especiales_miss,...,remainder__Num_Reservas_Previas_No_Canceladas,Duracion_Total,Total_Huespedes,Precio_por_Persona_Noche,Precio_Total_Estancia,SameDayBooking,Antelacion_Bin,Mes_sin,Mes_cos,Season_Bucket
0,4.060443,4.398146,1.0,0,0,0,0,0,0,1,...,0,2,2,2.199073,8.796292,0,1_1-7Days,1.000000,6.123234e-17,Primavera
1,5.645447,4.614625,0.0,0,0,0,0,0,1,0,...,0,3,2,2.307313,13.843876,0,1_1-7Days,-0.866025,5.000000e-01,Otoño
2,2.639057,4.382027,0.0,0,0,0,0,0,0,0,...,0,6,2,2.191013,26.292160,0,1_1-7Days,0.866025,5.000000e-01,Invierno


### 3.1 Validación de nuevas features

Comprobamos la existencia de todas las columnas generadas por el transformer de Feature Engineering:
- Duracion_Total  
- Total_Huespedes  
- Precio_por_Persona_Noche  
- Precio_Total_Estancia  
- SameDayBooking  
- Antelacion_Bin  
- Mes_sin  
- Mes_cos  
- Season_Bucket  

In [6]:
# Lista de features que debe crear FeatureCreator
expected_feats = [
    "Duracion_Total",
    "Total_Huespedes",
    "Precio_por_Persona_Noche",
    "Precio_Total_Estancia",
    "SameDayBooking",
    "Antelacion_Bin",
    "Mes_sin",
    "Mes_cos",
    "Season_Bucket"
]

# Verificamos su presencia en X_fe (salida de clean+FE)
for feat in expected_feats:
    print(f"{feat:25s} → {feat in X_fe.columns}")


Duracion_Total            → True
Total_Huespedes           → True
Precio_por_Persona_Noche  → True
Precio_Total_Estancia     → True
SameDayBooking            → True
Antelacion_Bin            → True
Mes_sin                   → True
Mes_cos                   → True
Season_Bucket             → True


## 4. Encoding y comprobación de tipos

A continuación extraemos el paso de encoding (pasos 0–2) y lo aplicamos, luego comprobamos que **todos** los dtypes sean numéricos o booleanos.

In [7]:
# Extraer clean → fe → encode
pre_enc = Pipeline(full_pipe.steps[:3], memory=None)

X_enc = pre_enc.fit_transform(X_sample, y_sample)

# Mostrar conteo de dtypes
print("Tipos de datos tras encoding:")
print(X_enc.dtypes.value_counts())

# Si quieres ver las primeras filas:
X_enc.head(3)

Dropped columns after FE: ['num_clean__Num_Adultos', 'num_clean__Num_Niños', 'num_clean__Noches_Semana', 'num_clean__Noches_Fin_Semana']
Tipos de datos tras encoding:
int64      17
float64    12
Name: count, dtype: int64


,target_enc__cat_clean__Tipo_Plan_Comidas,target_enc__cat_clean__Tipo_Habitación_Reservada,target_enc__cat_clean__Segmento_Mercado,target_enc__Antelacion_Bin,target_enc__Season_Bucket,remainder__num_clean__Tiempo_Antelación,remainder__num_clean__Precio_Promedio_Por_Habitación,remainder__num_clean__Num_Solicitudes_Especiales,remainder__num_clean__Num_Adultos_miss,remainder__num_clean__Num_Niños_miss,...,remainder__remainder__Huésped_Recurrente,remainder__remainder__Num_Cancelaciones_Previas,remainder__remainder__Num_Reservas_Previas_No_Canceladas,remainder__Duracion_Total,remainder__Total_Huespedes,remainder__Precio_por_Persona_Noche,remainder__Precio_Total_Estancia,remainder__SameDayBooking,remainder__Mes_sin,remainder__Mes_cos
0,0.156372,0.153522,0.216547,0.21212,0.205000,4.060443,4.398146,1.0,0,0,...,0,0,0,2,2,2.199073,8.796292,0,1.000000,6.123234e-17
1,0.156372,0.231722,0.219497,0.21212,0.196745,5.645447,4.614625,0.0,0,0,...,0,0,0,3,2,2.307313,13.843876,0,-0.866025,5.000000e-01
2,0.186208,0.231722,0.222401,0.21212,0.181055,2.639057,4.382027,0.0,0,0,...,0,0,0,6,2,2.191013,26.292160,0,0.866025,5.000000e-01


## 5. Conclusiones

- El pipeline de **limpieza + FE** genera las columnas esperadas.  
- Tras el **encoding**, todos los atributos son `int`, `float` o `bool`, listos para entrenar el modelo.  

En el próximo notebook (`03_Modeling.ipynb`) arrancaremos la fase de validación cruzada y entrenamiento final.